# Import Packages

In [1]:
import Strictly_Functions_Project as STP

from bisect import bisect_left
import sys
sys.path.append(r"C:\Users\jarra\Desktop\Masters\Heatwave_Project")
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import scipy
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import warnings
warnings.filterwarnings("ignore")
#RMSE 
from sklearn.metrics import mean_squared_error
from math import sqrt
from datetime import datetime
import PT13_Functions_For_Masters_New_Test as HW_Func


In [2]:
#

In [3]:
P_Gardens = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\perthgardens_daily_1880-1900.csv")
P_Gardens_Corr = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\perthgardens_daily_corrected_1880-1900.csv")

#Now we need to go back in time and minus 200 years off the date
#Convert To Datetime

P_Gardens['time'] = pd.to_datetime(P_Gardens['time'],format="%d/%m/%Y")
P_Gardens_Corr['time'] = pd.to_datetime(P_Gardens_Corr['time'],format="%d/%m/%Y")

#Split the Year up

P_Gardens = HW_Func.Date_Splitter(P_Gardens,'time',single= True)
P_Gardens_Corr = HW_Func.Date_Splitter(P_Gardens_Corr,'time',single= True)
P_Gardens['year'] = P_Gardens['year']-200
P_Gardens_Corr['year'] = P_Gardens_Corr['year']-200
#Combine in same format as ACORN-SAT

cols=["year","month","day"]
P_Gardens['date'] = P_Gardens[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
P_Gardens_Corr['date'] = P_Gardens_Corr[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")

del P_Gardens['time']
del P_Gardens_Corr['time']
del P_Gardens['year']
del P_Gardens_Corr['year']
del P_Gardens['month']
del P_Gardens_Corr['month']
del P_Gardens['day']
del P_Gardens_Corr['day']

P_Gardens['date'] = pd.to_datetime(P_Gardens['date'],format="%Y/%m/%d")
P_Gardens_Corr['date'] = pd.to_datetime(P_Gardens_Corr['date'],format="%Y/%m/%d")

#Max Temp, (drop(0) has dropped the 0th index, so it starts at 1)
MaxT_Perth = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\UPDATED TMAX, TMIN ACORN-SAT\tmax.009021.daily (3).csv").drop(0)
#Min Temp
MinT_Perth = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\UPDATED TMAX, TMIN ACORN-SAT\tmin.009021.daily (2).csv").drop(0)
#Ave Temp
AvgT_Perth = (MaxT_Perth['maximum temperature (degC)']+MinT_Perth['minimum temperature (degC)'])/2

Maximum = pd.Series(MaxT_Perth['maximum temperature (degC)'], name="Max")
Minimum = pd.Series(MinT_Perth['minimum temperature (degC)'],name="Min")
Average = pd.Series(AvgT_Perth,name="Avg")

#The Daily Max Min Ave Data
ACORN_SAT = pd.concat([MaxT_Perth['date'],Maximum,Minimum,Average],axis=1)
ACORN_SAT['date'] = pd.to_datetime(ACORN_SAT['date'],format="%Y/%m/%d")



#Now load in and fix Perth Gardens 1830-1875
Per_Gard = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\swanriver_subdaily_1830-1875.csv")
Per_Gard
#Set Datetime
Per_Gard['date'] = pd.to_datetime(Per_Gard['date'],dayfirst = True)
    

#So this works, now we need to expand this into a for loop and make this a new dataframe to be added onto
#the historical time because it will make things easier



#Can't set index yet.
#Perth Regional Office 1967 to 1992 sub daily dataset
PRO_Sub = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\perthregionaloffice_subdaily_1942-1992.csv")

PRO_Sub['date'] = pd.to_datetime(PRO_Sub['date'],dayfirst = True)
PRO_Sub = PRO_Sub.set_index('date')
PRO_Sub =PRO_Sub['temp']/10 

PRO_Sub_ES  = PRO_Sub.loc['1967':'1992']
PRO_Sub_ES =PRO_Sub_ES

## Perth Regional Office Daily Extreme Dataset
#Load PRO in
#BOM PERTH REGIONAL OFFICE
MaxT_PRO = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\IDCJAC0010_009034_1800_Data.csv")
MinT_PRO = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\IDCJAC0011_009034_1800_Data.csv")

#Clean The data
MaxT_PRO['Datetime']= pd.to_datetime(MaxT_PRO[['Year', 'Month', 'Day']])
MinT_PRO['Datetime']= pd.to_datetime(MinT_PRO[['Year', 'Month', 'Day']])

#Delete irrelevent columns
del MaxT_PRO['Product code']
del MaxT_PRO['Bureau of Meteorology station number']
del MaxT_PRO['Year']
del MaxT_PRO['Month']
del MaxT_PRO['Day']
del MaxT_PRO['Days of accumulation of maximum temperature']
del MaxT_PRO['Quality']
del MinT_PRO['Product code']
del MinT_PRO['Bureau of Meteorology station number']
del MinT_PRO['Year']
del MinT_PRO['Month']
del MinT_PRO['Day']
del MinT_PRO['Days of accumulation of minimum temperature']
del MinT_PRO['Quality']

#Change the column name to date
MaxT_PRO= MaxT_PRO.rename(columns={'Datetime':'date'})
MinT_PRO= MinT_PRO.rename(columns={'Datetime':'date'})


#Change the column names
MaxT_PRO= MaxT_PRO.rename(columns={'Maximum temperature (Degree C)':'PRO Max'})
MinT_PRO= MinT_PRO.rename(columns={'Minimum temperature (Degree C)':'PRO Min'})

#Now concat it
MaxT_PRO= MaxT_PRO.set_index('date')
MinT_PRO= MinT_PRO.set_index('date')

PRO_DE = pd.merge(left = MaxT_PRO,right  =MinT_PRO,left_index=True,right_index=True  )

#Training Data
PRO_DE_Training = PRO_DE.loc['1967':'1992']
PRO_Sub_Training = PRO_Sub_ES
#Estimating Data
PRO_Sub_4_Est = PRO_Sub.reset_index()
T_Ext =PRO_DE_Training.reset_index()
T_Sub =PRO_Sub_ES.reset_index()
S_Est = PRO_Sub_4_Est


#Get the Max and Min Information
MaxT_Perth = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\tmax.009021.daily.csv").drop(0).reset_index(drop=True)
MinT_Perth = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\tmin.009021.daily.csv").drop(0).reset_index(drop=True)

#Rename the columns
Maximum = pd.Series(MaxT_Perth['maximum temperature (degC)'], name="Max")
Minimum = pd.Series(MinT_Perth['minimum temperature (degC)'],name="Min")

#Concat it all together
Daily_MaxMin = pd.concat([MaxT_Perth['date'],Maximum,Minimum],axis=1)

#Apply datetime
Daily_MaxMin['date'] = pd.to_datetime(Daily_MaxMin['date'],format="%d/%m/%Y")

#Dates 
#This is used in the concatination process when the CDP is developed and other things 
#that have the data disappear. Since the full 366 days need to be accounted for, 2020 was 
#the year I chose for this
#'''
Dates = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\Dates, includes feb 29.csv")


In [6]:
Data_1830_1875_All = STP.Temp_Estimation(Per_Gard, T_Sub, T_Ext, 1)
#Extract TriaL 1
Data_1830_1875 = Data_1830_1875_All.get("Trial_1")

#Delete The correlations
del Data_1830_1875['Correlation Max T']
del Data_1830_1875['Correlation Min T']

#rename
Data_1830_1875 = Data_1830_1875.rename(columns={'Max Temp Estimation': 'Max','Min Temp Estimation': 'Min' })
Data_1830_1875 = Data_1830_1875.reset_index()
Data_1830_1875 
#

,date,Max,Min
0,1830-04-16,27.007308,NaN
1,1830-04-17,27.596920,14.997224
2,1830-04-18,29.487644,18.325167
3,1830-04-19,28.430873,19.324581
4,1830-04-20,26.218648,12.510398
...,...,...,...
16692,1875-12-28,39.010538,NaN
16693,1875-12-29,36.075399,22.561407
16694,1875-12-30,33.140260,18.517799
16695,1875-12-31,27.374808,19.369610


In [8]:
#ACORN_SAT_1910_2021
Daily_MaxMin = Daily_MaxMin.set_index('date')



In [9]:



# Create a new dataframe with a column for every day between 1835-04-16 and 2021-01-01
start_date = pd.to_datetime('1830-04-16')
end_date = pd.to_datetime('2021-06-30')
date_range = pd.date_range(start_date, end_date, freq='D')
df_dates = pd.DataFrame({'date': date_range})
df_dates
# Merge the new dataframe with the two existing dataframes using a left join
df_merged = pd.merge(df_dates, Data_1830_1875[['date', 'Max', 'Min']], on='date', how='left')
#df_merged = pd.merge(df_merged[['date', 'Max', 'Min']], Daily_MaxMin[['date', 'Max', 'Min']], on='date', how='left')
df_merged = df_merged.set_index('date')
# Fill any missing values with the last known value (forward fill)
#df_merged.fillna(method='ffill', inplace=False)
df_concat = pd.concat([df_merged,Daily_MaxMin],axis =0)
df_concat = df_concat.reset_index()
df_concat = df_concat.drop_duplicates(subset=["date"], keep='last')
df_concat = df_concat.set_index('date')
df_concat.reset_index()

,date,Max,Min
0,1830-04-16,27.007308,NaN
1,1830-04-17,27.596920,14.997224
2,1830-04-18,29.487644,18.325167
3,1830-04-19,28.430873,19.324581
4,1830-04-20,26.218648,12.510398
...,...,...,...
69833,2021-06-26,20.300000,5.800000
69834,2021-06-27,15.600000,8.700000
69835,2021-06-28,19.700000,10.500000
69836,2021-06-29,18.000000,12.500000


In [ ]:
df_concat = df_concat.

In [11]:

Heatwaves,CDP = STP.Heatwave_Function_v4(df_concat,
                         Dates,
                         CDP_Matrix = [],
                         Heatwave_Detail= True,
                         Percentile = 85,
                         window = 7,
                         CDP_start_end_years = [1961,1990])

AttributeError: Can only use .dt accessor with datetimelike values